In [14]:
# Import the required libraries.
# !pip install tensorflow
import os
import cv2

import numpy as np

import tensorflow as tf
import keras

import matplotlib.pyplot as plt

# from moviepy.editor import *
%matplotlib inline
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model
import warnings
warnings.filterwarnings('ignore')

In [15]:
from keras.applications.vgg16 import VGG16

In [16]:
model= VGG16()

In [17]:
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [18]:
deepface_model = tf.keras.models.Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)


In [19]:
deepface_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [20]:
def normalize(x):
    return x / np.sqrt(np.sum(np.multiply(x, x)))
def findEuclideanDistance(source_representation, test_representation):
    euclidean_distance = source_representation - test_representation
    euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
    euclidean_distance = np.sqrt(euclidean_distance)
    return euclidean_distance

In [21]:
def process_img(image_path):
    test_image=tf.keras.preprocessing.image.load_img(image_path,target_size=(224,224))
    test_image=tf.keras.preprocessing.image.img_to_array(test_image)
    test_image=np.expand_dims(test_image,axis=0)
    return test_image

In [11]:
# Embedding_List = [] # list of embeding matrix after conversion from cropped faces to embedding matrix using deepface
# Name_List = []  # list of names corrospoing to cropped photos
# for folderPath in os.listdir('dataset'):
#     fpath=os.path.join('dataset',folderPath) #traversing through all folders in the dataset
#     for imgpath in os.listdir(fpath): #traversing through all images in the sub folders of dataset
#         imagepath=os.path.join(fpath,imgpath)
#         Embedding_List.append(deepface_model.predict(process_img(imagepath))[0])
#         Name_List.append(folderPath)

# #saving data
# # data = [Name_List, Embedding_List]

1/1 [==============================] - 0s 149ms/step


In [15]:
# import numpy as np

# # Assuming your array is named 'arr'
# np.save('embeddings.npy', Embedding_List)

In [16]:
# np.save('name_list.npy',Name_List)

In [22]:
name_list = np.load('name_list.npy')

In [23]:
embedding_list = np.load('embeddings.npy')

In [24]:
def predict(image):
    dist_list = [] #making distance list array
    R_embedding = deepface_model.predict(process_img(image))[0]
    for emb_db in embedding_list:
        euclidean_distance = findEuclideanDistance(normalize(R_embedding),normalize(emb_db))
        dist_list.append(euclidean_distance)
    min_dist = min(dist_list) #finding min distance
    print(min_dist)
    if min_dist<0.70: #condition for surity, this value is selected by us by oberving its value for differnet persons
        min_dist_idx = dist_list.index(min_dist)
        name = name_list[min_dist_idx]
        return name
    else:
        name  = 'unknown'
        return name

In [25]:
import cv2
from pathlib import Path

open_door = False
cascade_path = cv2.data.haarcascades + "haarcascade_frontalface_default.xml" # Path to the Haar Cascade classifier XML file

# Load the cascade
face_cascade = cv2.CascadeClassifier(cascade_path)

cam = cv2.VideoCapture(0)

while True:
    ret, frame = cam.read()

    if not ret:
        print("No image error...")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert image to grayscale

   
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 1)  # Draw rectangle around the face

     
        crop_img = frame[y:y + h, x:x + w]
        Path('Test_face/').mkdir(parents=True, exist_ok=True)
        cv2.imwrite('Test_face/newface.jpg', crop_img)
        R_ImagePath = 'Test_face/newface.jpg'

        
        predicted_name = predict(R_ImagePath)
        cv2.putText(frame, predicted_name, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2,
                    cv2.LINE_AA)  # Write name of the predicted person above the rectangle

    cv2.imshow('video', frame)

    # Keypress handling
    key = cv2.waitKey(1)

    if key == 32:
        # Space pressed
        print("Analysing Person")
        person = predicted_name

    elif key == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break

cam.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 418ms/step
0.99874425
1/1 [==============================] - 0s 267ms/step
1.020548
1/1 [==============================] - 0s 160ms/step
0.9935083
1/1 [==============================] - 0s 178ms/step
1.1776379
1/1 [==============================] - 0s 176ms/step
0.80102813
1/1 [==============================] - 0s 203ms/step
0.78453517
1/1 [==============================] - 0s 206ms/step
1.1341984
1/1 [==============================] - 0s 161ms/step
0.7569395
1/1 [==============================] - 0s 162ms/step
0.7408798
1/1 [==============================] - 0s 169ms/step
0.7026921
1/1 [==============================] - 0s 229ms/step
0.6846208
1/1 [==============================] - 0s 198ms/step
0.6666758
1/1 [==============================] - 0s 145ms/step
0.6347586
1/1 [==============================] - 0s 250ms/step
1.0708425
1/1 [==============================] - 0s 162ms/step
0.68432176
1/1 [==============================] - 0s 208ms/step

1/1 [==============================] - 0s 161ms/step
0.5830025
1/1 [==============================] - 0s 208ms/step
0.5661061
1/1 [==============================] - 0s 224ms/step
0.5649178
1/1 [==============================] - 0s 177ms/step
0.56945336
1/1 [==============================] - 0s 159ms/step
0.5861824
1/1 [==============================] - 0s 192ms/step
0.5377995
1/1 [==============================] - 0s 155ms/step
0.54421765
1/1 [==============================] - 0s 161ms/step
0.52899003
1/1 [==============================] - 0s 229ms/step
0.5249114
1/1 [==============================] - 0s 225ms/step
0.51979834
1/1 [==============================] - 0s 142ms/step
0.5177523
1/1 [==============================] - 0s 161ms/step
0.5141799
1/1 [==============================] - 0s 145ms/step
0.53621113
1/1 [==============================] - 0s 175ms/step
0.5634052
1/1 [==============================] - 0s 162ms/step
0.55841875
1/1 [==============================] - 0s 145ms/s